In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import gmaps
import requests
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("../weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bethel,US,41.3712,-73.4140,57.81,67,75,6.91,broken clouds
1,1,Gardiner,US,44.2301,-69.7753,53.08,73,1,0.00,clear sky
2,2,Nanortalik,GL,60.1432,-45.2371,35.87,69,34,5.79,scattered clouds
3,3,Ribeira Grande,PT,38.5167,-28.7000,67.32,68,21,13.13,few clouds
4,4,Kizukuri,JP,40.8061,140.3861,50.95,72,97,8.10,overcast clouds


In [22]:
# Prompt the customer for the minimum temperature preference.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
# Prompt the customer for the maximum temperature preference. 
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [23]:
preferred_cities_df = city_data_df.loc[
                                       (city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [24]:
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Puerto Palomas,MX,31.7667,-107.6167,79.93,16,1,23.31,clear sky
8,8,Kununurra,AU,-15.7667,128.7333,76.98,94,2,5.75,clear sky
12,12,Vaini,TO,-21.2000,-175.2000,77.16,61,20,19.57,few clouds
14,14,Maragogi,BR,-9.0122,-35.2225,79.11,80,29,11.30,scattered clouds
15,15,Hithadhoo,MV,-0.6000,73.0833,82.31,74,60,12.88,broken clouds
17,17,San Patricio,US,28.0170,-97.5169,88.52,53,14,15.43,few clouds
23,23,Tual,ID,-5.6667,132.7500,83.73,75,94,14.41,overcast clouds
27,27,Sungairaya,ID,0.7000,108.9000,76.42,87,71,3.69,broken clouds
30,30,Acapulco,MX,16.8634,-99.8901,75.02,88,90,16.11,overcast clouds
32,32,Kapaa,US,22.0752,-159.3190,82.38,75,40,4.00,scattered clouds


In [25]:
preferred_cities_df.count()


City_ID                219
City                   219
Country                218
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [26]:
preferred_cities_df = preferred_cities_df.dropna(how='any')

In [27]:
preferred_cities_df.count()

City_ID                218
City                   218
Country                218
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Puerto Palomas,MX,79.93,clear sky,31.7667,-107.6167,
8,Kununurra,AU,76.98,clear sky,-15.7667,128.7333,
12,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
14,Maragogi,BR,79.11,scattered clouds,-9.0122,-35.2225,
15,Hithadhoo,MV,82.31,broken clouds,-0.6000,73.0833,
17,San Patricio,US,88.52,few clouds,28.0170,-97.5169,
23,Tual,ID,83.73,overcast clouds,-5.6667,132.7500,
27,Sungairaya,ID,76.42,broken clouds,0.7000,108.9000,
30,Acapulco,MX,75.02,overcast clouds,16.8634,-99.8901,
32,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,


In [29]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Puerto Palomas,MX,79.93,clear sky,31.7667,-107.6167,Hotel San Carlos
8,Kununurra,AU,76.98,clear sky,-15.7667,128.7333,Hotel Kununurra
12,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
14,Maragogi,BR,79.11,scattered clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
15,Hithadhoo,MV,82.31,broken clouds,-0.6000,73.0833,Scoop Guest House
17,San Patricio,US,88.52,few clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
23,Tual,ID,83.73,overcast clouds,-5.6667,132.7500,Grand Vilia Hotel
27,Sungairaya,ID,76.42,broken clouds,0.7000,108.9000,Mess P.Milu
30,Acapulco,MX,75.02,overcast clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
32,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [40]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations_df = hotel_df[["Lat", "Lng"]]
locations_df.head()


,Lat,Lng
5,31.7667,-107.6167
8,-15.7667,128.7333
12,-21.2000,-175.2000
14,-9.0122,-35.2225
15,-0.6000,73.0833


In [57]:
# 4a. Add a marker layer for each city to the map.
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [72]:
vacation_start = hotel_df.loc[hotel_df['City'] == 'Gamba']
vacation_end = hotel_df.loc[hotel_df['City'] == 'Gamba']
vacation_stop1 = hotel_df.loc[hotel_df['City'] == 'Buchanan']
vacation_stop2 = hotel_df.loc[hotel_df['City'] == 'Umm Kaddadah'] 
vacation_stop3 = hotel_df.loc[hotel_df['City'] == 'Uvira']

In [73]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start['Lat'].to_numpy()[0], vacation_start['Lng'].to_numpy()[0]
end = vacation_end['Lat'].to_numpy()[0], vacation_end['Lng'].to_numpy()[0]
stop1 = vacation_stop1['Lat'].to_numpy()[0], vacation_stop1['Lng'].to_numpy()[0]
stop2 = vacation_stop2['Lat'].to_numpy()[0], vacation_stop2['Lng'].to_numpy()[0]
stop3 = vacation_stop3['Lat'].to_numpy()[0], vacation_stop3['Lng'].to_numpy()[0]

In [76]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
travel_route = gmaps.directions_layer(start, end, waypoints=[stop1,stop2,stop3] travel_mode="DRIVING")
fig.add_layer(travel_route)
fig

AssertionError: 